<a href="https://colab.research.google.com/github/bluecityisu/class2022/blob/main/cate_%E4%B8%AD%E6%96%87%E7%9A%84%E8%AA%9E%E8%A8%80%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
import string
import re
import csv
import numpy as np

In [ ]:
from bs4 import BeautifulSoup
table=str.maketrans('', '', string.punctuation)

In [ ]:
stopwords = ["我","了"]

In [ ]:
def getch(s):
  str33=[]
  res = re.compile(r"([\u4e00-\u9fa5])")
  ret = res.split(s)
  for ch in ret:
    if ch!='' and ch!=' ':
      str33.append(ch)
  str33=np.array(str33)
  return str33

In [ ]:
sentences=[]
labels=[]
with open('/content/drive/MyDrive/TensorFlow_Book/Deep_learning_coder/ch3.csv', encoding='UTF-8') as csvfile:
  reader = csv.reader(csvfile, delimiter=",")
  for row in reader:
    labels.append(row[0])
    sentence = row[1].lower()
    sentence = sentence.replace("，", ",")
    sentence = sentence.replace("?", "?")
    sentence = sentence.replace("。", ".")
    sentence = sentence.replace("/", " / ")
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()

    #words = sentence.split()
    words=getch(sentence)
    filtered_sentence = ""
    for word in words:
        word = word.translate(table)
        if word not in stopwords:
            filtered_sentence = filtered_sentence + word
    sentences.append(filtered_sentence)

print(len(labels))
print(len(sentences))
print(sentences[4])

In [ ]:
sentences[0]

In [ ]:
vocab_size = 200
embedding_dim = 6
max_length = 10
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [ ]:
sent2=[]
for nx in sentences:
  nn2=getch(nx)
  nn2=nn2.flatten()
  sent2.append(nn2)

In [ ]:
sent2[0]

In [ ]:
sent2=np.array(sent2)
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
for xxx in sent2:
  tokenizer.fit_on_texts(xxx)

word_index = tokenizer.word_index

In [ ]:
print(word_index)

In [ ]:
sentences

In [ ]:
mylist=[]

for nn in sentences:
  nn2=getch(nn)
  nn2 = tokenizer.texts_to_sequences(nn2)
  nn2=np.array(nn2)
  #padd = pad_sequences(str3, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  nn2=nn2.flatten()
  mylist.append(nn2)
mylist=np.array(mylist,dtype=object)
print(mylist[3])
mylist = pad_sequences(mylist, maxlen=max_length, padding=padding_type, truncating=trunc_type)
mylist


In [ ]:
training_labels=[x.replace(" ", "") for x in labels]

In [ ]:
training_labels[0]=0

In [ ]:
for x in range(len(training_labels)):
  if training_labels[x]=='0':
    training_labels[x]=0
  elif training_labels[x]=='1':
    training_labels[x]=1
  elif training_labels[x]=='2':
    training_labels[x]=2
  else:
    training_labels[x]=int(training_labels[x])


In [ ]:
training_labels=np.array(training_labels)

In [ ]:
print(training_labels)

In [ ]:
ys = tf.keras.utils.to_categorical(training_labels, num_classes=3)

In [ ]:
print(len(mylist))
print(len(ys))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(LSTM(max_length-1, return_sequences=True)),
    tf.keras.layers.Bidirectional(LSTM(max_length-1)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 100
history = model.fit(mylist, ys, epochs=num_epochs, verbose=2)


In [ ]:
cate=["高興","生氣","發問"]
mylist=[]
nn3=[]
str3 = ["保存期限還有多久?","品質優良"]
str3=np.array(str3)
for xx,yy in enumerate(str3):
  str3[xx]=yy.replace("?","")

str3=np.array(str3)

for nn in str3:
  nn2=getch(nn)
  nn2 = tokenizer.texts_to_sequences(nn2)
  nn2=np.array(nn2)
  #padd = pad_sequences(str3, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  nn2=nn2.flatten()
  mylist.append(nn2)
mylist=np.array(mylist)
print(mylist)
mylist = pad_sequences(mylist, maxlen=max_length, padding=padding_type, truncating=trunc_type)


predict=model.predict(mylist)
#print(pred_classes.reshape(-1)[predicted])
print(np.round(predict,2))
print(np.argmax(predict,axis=1))
result=[cate[x] for x in np.argmax(predict,axis=1)]
print(result)

In [ ]:
def preds(x):
  mylist=[]
  nn3=[]
  str3=x.replace("?","")


  nn2=getch(str3)
  nn2 = tokenizer.texts_to_sequences(nn2)
  nn2=np.array(nn2)
    #padd = pad_sequences(str3, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  nn2=nn2.flatten()
  mylist.append(nn2)
  mylist=np.array(mylist)
  mylist = pad_sequences(mylist, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  print(mylist)

  predict=model.predict(mylist)
  print(np.round(predict,2))
  pred_index=np.argmax(predict,axis=1)
  print(pred_index)
  return cate[pred_index[0]]

In [ ]:
print(preds("品質優良"))